# Computing Optimal Enhancement Order for Hayato's 6th Job
### Soliciting community crowdsourcing of correct BA proportions for full and burst (Kalos?) rotations.

In [540]:
import numpy as np
import copy

'''
Hayato 6th job Skill Strength Increases (Enhancement [5th job], Mastery [4th job], and Origin Skill Cores)
'''

# more info: https://www.reddit.com/r/Maplestory/comments/17htmz3/new_age_patch_breakdown_part_2_the_6th_job/
'''
5th job enhancement strength array 
levels 1-30, from lvl 1-9:11-19% fd, lvl 10-19: 25-34% fd, 
lvl 20-29: 40-49% fd, lvl30 = 60% fd
additive FD, level 1 starts at 1.11 multiplier, then 1.12, 1.13, ..., 1.60
'''
enh_5th = np.array([
         0.11, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, # level 1-9 = 19fd
         0.06, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, # level 10 - 19 = 34fd
         0.06, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, # level 20 - 29 = 49fd
         0.11 ]) # level 30 = 60fd

'''
origin enhancement strength array
per level increase: first slash: 30%, 3 slices: 20% each, last slash: 80%
level 1:  (1130 * 8 * 10 + 920  * 10 * 15 + 2480 * 15 * 10) = 600400% damage
level 2:  (1160 * 8 * 10 + 940  * 10 * 15 + 2560 * 15 * 10) = 17400% damage per level
level 30: (2000 * 8 * 10 + 1500 * 10 * 15 + 4800 * 15 * 10) = 1105000% damage = 84% fd
origin skill has additional bonuses at lvl 10, 20, 30
10: 20 ied, 20: bd, 30: 30 ied, 30 bd -- calculated as placeholder FD
this array is applied multiplicatively, not additively, unlike enh_5th
'''
enh_ori = np.array([(600400 + i*17400)/(600400 + (i-1)*17400) for i in range (1, 31)])
ori_dict = {10: 1.01, 20: 1.025, 30: 1.04} # spitball fd values, applied multiplicatively
for index, value in ori_dict.items():
    enh_ori[index-1] = enh_ori[index-1] * value

'''
4th job mastery enhancement strength array
level 0->1:   354 -> 366%
level 1->2:   366 -> 372% = 6% damage per level
level 29->30: 540 -> 546% = 50% fd total
this array is applied multiplicatively, not additively
'''
enh_4th = np.array([((366 + i*6)/(366 + (i-1)*6)) for i in range (1, 31)])
enh_4th[0] = 1.0547 # 366/347 (lvl21 rbf) = 1.0547



In [541]:
enh_4th.round(4)

array([1.0547, 1.0161, 1.0159, 1.0156, 1.0154, 1.0152, 1.0149, 1.0147,
       1.0145, 1.0143, 1.0141, 1.0139, 1.0137, 1.0135, 1.0133, 1.0132,
       1.013 , 1.0128, 1.0127, 1.0125, 1.0123, 1.0122, 1.012 , 1.0119,
       1.0118, 1.0116, 1.0115, 1.0114, 1.0112, 1.0111])

In [542]:
enh_ori.round(4)

array([1.029 , 1.0282, 1.0274, 1.0267, 1.026 , 1.0253, 1.0247, 1.0241,
       1.0235, 1.0332, 1.0225, 1.022 , 1.0215, 1.0211, 1.0206, 1.0202,
       1.0198, 1.0194, 1.019 , 1.0442, 1.0183, 1.018 , 1.0177, 1.0174,
       1.0171, 1.0168, 1.0165, 1.0163, 1.016 , 1.0564])

In [543]:
# enhancement cost matrix
# level 2-30
origin_cost      = np.array([(1, 30),  (1, 35),  (1, 40),  (2, 45),   (2, 50),
                             (2, 55),  (3, 60),  (3, 65),  (10, 200), (3, 80),
                             (3, 90),  (4, 100), (4, 110), (4, 120),  (4, 130),
                             (4, 140), (4, 150), (5, 160), (15, 350), (5, 170),  
                             (5, 180), (5, 190), (5, 200), (5, 210),  (6, 220),  
                             (6, 230), (6, 240), (7, 250), (20, 500)])
# level 1-30
mastery_cost     = np.array([(3, 50),  (1, 15),  (1, 18),  (1, 20),  (1, 23),
                             (1, 25),  (2, 28),  (2, 30),  (2, 33),  (5, 100),
                             (2, 40),  (2, 45),  (2, 50),  (2, 55),  (2, 60),
                             (2, 65),  (2, 70),  (2, 75),  (3, 80),  (8, 175),
                             (3, 85),  (3, 90),  (3, 95),  (3, 100), (3, 105),
                             (3, 110), (3, 115), (3, 120), (4, 125), (10, 250)])
# level 1-30
enhancement_cost = np.array([(4, 75),  (1, 23),  (1, 27),  (1, 30),  (2, 34),
                             (2, 38),  (2, 42),  (3, 45),  (3, 49),  (8, 150),
                             (3, 60),  (3, 68),  (3, 75),  (3, 83),  (3, 90),
                             (3, 98),  (3, 105), (4, 113), (4, 120), (12, 263),
                             (4, 128), (4, 135), (4, 143), (4, 150), (4, 158),
                             (5, 165), (5, 173), (6, 180), (6, 188), (15, 375)])

In [544]:
# Full Rot and Burst BA proportions in percentages
# taken from Labels' BA
# assuming unleveled enhancement cores (lvl 0, no fd)
ba_prop    = {"zankou" :0.110,
              "slice"  :0.079,
              "jamal"  :0.067,
              "pb"     :0.064,
              "jinqd"  :0.084, # origin skill
              "rbf"    :0.336}

burst_prop = {"zankou" :0.139,
              "slice"  :0.076,
              "jamal"  :0.106,
              "pb"     :0.073,
              "jinqd"  :0.281, # origin skill
              "rbf"    :0.137}

# dictionary to keep track of current upgrade level of each skill
# key: skill name, value: level (corresponds to index of enh_5th)
skills = {"zankou": 0, 
          "slice":  0, 
          "jamal":  0, 
          "pb":     0, 
          "jinqd":  1, # idk how origin skill damage increase works
          "rbf":    0}

## "Greedy" Search for Upgrade Order
### Computation Methodology
There are 6 skills we can boost currently: `Rai Blade Flash` (Mastery Core [4th Job]), `Jin Quick Draw` (Origin Core [6th Job]), and `Instant Slice, Zankou, Phantom Blade, and Susano-o` (Enhancement Core [5th Job]).

In the BA's provided by Labels, these 6 skills each make up some % (let's call them `proportions`) of the BA (total 100%, or `1.0`). In the first step, the algorithm picks RBF to upgrade first. Why? RBF unboosted makes up `0.336` (33.6%) of a full rotation BA. Leveling its Mastery Core from 0-->1 is a 5.47% fd increase (366% damage level 1, 347% damage unboosted) and provides an absolute `0.018` (1.8%) increase to the BA. Its upgrade cost is 3 Sol Erda and 50 fragments, which I calculate as (`3 * energy + 1 * fragments`). Feel free to give me a better cost ratio. The strength_gain:cost ratio (`0.018/350`) is the highest of the 6 skills, so it is picked. 

As a result, RBF's new proportion in a BA would be `0.354` out of `1.018` (`1.0 + 0.018`). Then, in order to normalize the `proportions` to be percentages, we recalculate the initial `proportions` to be out of this new `1.018` value (let's call it `total_strength`). 

This process iterates until all skills have been fully upgraded.

NOTE: All the hardcoded proportions are subject to change when 6th job lands and GMS people do BAs. This is a greedy algorithm that only looks at the immediate damage gain and immediate cost. We can tune these costs and heuristics. This is especially important because the amortized FD gain spikes as you approach levels 10, 20, and 30.

In [545]:
def cost_fn(skill, level, strategy="default"):
    '''
    returns the cost of upgrading a skill to a certain level
    '''
    energy, fragments = 0, 0
    if skill == "jinqd":
        energy, fragments = origin_cost[level-1]
    elif skill == "rbf":
        energy, fragments = mastery_cost[level]
    else:
        energy, fragments = enhancement_cost[level]
    return (100*energy) + fragments

# TODO: heuristic fn
#  lookahead to next 3? levels to see how much fd is coming up. specified by strategy flag
def heuristic_fn(skill, level, strategy="default"):
    pass

In [546]:
# Function to calculate overall strength, skipping discretized steps of greedy search
# basically a validation function
def calculate_strength(fd_multipliers, origin_multipliers):
    temp_fd = copy.deepcopy(ba_prop)
    for skill_name, pct in ba_prop.items():
        if skill_name == "jinqd":
            temp_fd[skill_name] *= 2.01
        elif skill_name == "rbf":
            temp_fd[skill_name] *= 1.54
        else:
            temp_fd[skill_name] *= 1.6
    # sum of all fd * proportion of each skill
    increase = (np.array(list(temp_fd.values())) - np.array(list(ba_prop.values()))).sum()
    return  (1 + increase / 1)

In [547]:
order = [] # order of upgrades
total_strength_gain = 1.0
temp_prop   = copy.deepcopy(ba_prop)
temp_skills = copy.deepcopy(skills)
# Brute force search
for iteration in range(180):
# for iteration in range(20):
    print(f"Upgrade: {iteration} total_strength_gain: {total_strength_gain}")
    max_strength_increase = 0
    max_strength_increase_ratio = 0
    selected_skill = None
    new_prop = 0

    # Iterate through each skill
    for skill_name, level in temp_skills.items():
        # Forecast one level ahead for each skill
        level = int(level)
        next_level = int(level + 1)
        # Check if the skill has reached the maximum level
        if next_level > len(enh_5th):
            continue

        # Calculate proportional increase in strength
        # strength_increase = (total_strength_gain + (ngain * orig_prop) - prop) / (total_strength_gain)
        cost = 0
        cprop = 0 # current contribution of total damage (absolute value, not proportion)
        nprop = 0 # new contribution of total damage
        orig_prop = ba_prop[skill_name] # original proportion
        if skill_name == "jinqd":
            cost = cost_fn(skill_name, level)
            cprop = enh_ori[:level].prod() * orig_prop
            nprop = enh_ori[:next_level].prod() * orig_prop
        elif skill_name == "rbf":
            cost = cost_fn(skill_name, level)
            cprop = enh_4th[:level].prod() * orig_prop
            nprop = enh_4th[:next_level].prod() * orig_prop
        else:
            cost = cost_fn(skill_name, level)
            cprop = (1 + enh_5th[:level].sum()) * orig_prop
            nprop = (1 + enh_5th[:next_level].sum()) * orig_prop

        strength_increase = nprop - cprop
        strength_cost_ratio = (strength_increase*1000)/cost
        print(f"skill name: ({level}) {skill_name:>6}, nprop: {np.round(nprop, 3):>5}, cprop: {np.round(cprop,3):>5}, strength_increase: {np.round(strength_increase,4):>6}, cost: {cost:>4}, ratio: {np.round(strength_cost_ratio, 4)}")
        
        # Check if this is the maximum increase so far
        # if strength_increase > max_strength_increase:
        if strength_cost_ratio > max_strength_increase_ratio:
            max_strength_increase = strength_increase
            max_strength_increase_ratio = strength_cost_ratio
            selected_skill = skill_name
            new_prop = nprop
            # recalculating total strength gain

    # Apply the selected enhancement
    if selected_skill is not None:
        temp_skills[selected_skill] += 1 # increase level by 1
        # Increment total strength gain and update the skill damage proportions
        total_strength_gain += max_strength_increase
        for skill_name, prop in temp_prop.items():
            temp_prop[skill_name] = prop/total_strength_gain
        temp_prop[selected_skill] = new_prop/total_strength_gain
        update_str = f"Upgrade #: {iteration + 1:>3}, Skill: {selected_skill:>6}, New Level: {temp_skills[selected_skill]:>2}, max_strength_increase: {round(max_strength_increase, 4):>5}, max_strength_increase_ratio: {round(max_strength_increase_ratio, 4)}"
        print(update_str)
        order.append(update_str)

# After the loop, you can print the final results
final_strength = calculate_strength(dict, enh_ori)
print("Final Strength (validation):", final_strength)
print(f"temp_prop: {temp_prop}")

Upgrade: 0 total_strength_gain: 1.0
skill name: (0) zankou, nprop: 0.122, cprop:  0.11, strength_increase: 0.0121, cost:  475, ratio: 0.0255
skill name: (0)  slice, nprop: 0.088, cprop: 0.079, strength_increase: 0.0087, cost:  475, ratio: 0.0183
skill name: (0)  jamal, nprop: 0.074, cprop: 0.067, strength_increase: 0.0074, cost:  475, ratio: 0.0155
skill name: (0)     pb, nprop: 0.071, cprop: 0.064, strength_increase:  0.007, cost:  475, ratio: 0.0148
skill name: (1)  jinqd, nprop: 0.089, cprop: 0.086, strength_increase: 0.0024, cost:  130, ratio: 0.0187
skill name: (0)    rbf, nprop: 0.354, cprop: 0.336, strength_increase: 0.0184, cost:  350, ratio: 0.0525
Upgrade #:   1, Skill:    rbf, Selected Level:  1, max_strength_increase: 0.0184, max_strength_increase_ratio: 0.0525
Upgrade: 1 total_strength_gain: 1.0183792
skill name: (0) zankou, nprop: 0.122, cprop:  0.11, strength_increase: 0.0121, cost:  475, ratio: 0.0255
skill name: (0)  slice, nprop: 0.088, cprop: 0.079, strength_increase

In [548]:
total_strength_gain

1.4587721686927015

In [549]:
order

['Upgrade #:   1, Skill:    rbf, Selected Level:  1, max_strength_increase: 0.0184, max_strength_increase_ratio: 0.0525',
 'Upgrade #:   2, Skill:    rbf, Selected Level:  2, max_strength_increase: 0.0057, max_strength_increase_ratio: 0.0497',
 'Upgrade #:   3, Skill:    rbf, Selected Level:  3, max_strength_increase: 0.0057, max_strength_increase_ratio: 0.0484',
 'Upgrade #:   4, Skill:    rbf, Selected Level:  4, max_strength_increase: 0.0057, max_strength_increase_ratio: 0.0476',
 'Upgrade #:   5, Skill:    rbf, Selected Level:  5, max_strength_increase: 0.0057, max_strength_increase_ratio: 0.0465',
 'Upgrade #:   6, Skill:    rbf, Selected Level:  6, max_strength_increase: 0.0057, max_strength_increase_ratio: 0.0457',
 'Upgrade #:   7, Skill: zankou, Selected Level:  1, max_strength_increase: 0.0121, max_strength_increase_ratio: 0.0255',
 'Upgrade #:   8, Skill:    rbf, Selected Level:  7, max_strength_increase: 0.0057, max_strength_increase_ratio: 0.0251',
 'Upgrade #:   9, Skill:

## Scraps

In [550]:
import base64
import requests

# OpenAI API Key
api_key = "API_KEY"

# Function to encode the image
def encode_image(image_path):
  with open(image_path, "rb") as image_file:
    return base64.b64encode(image_file.read()).decode('utf-8')

# Path to your image
image_path = "hexa-matrix-sol-erda-costs-2.png"

# Getting the base64 string
base64_image = encode_image(image_path)

headers = {
  "Content-Type": "application/json",
  "Authorization": f"Bearer {api_key}"
}

payload = {
  "model": "gpt-4-vision-preview",
  "messages": [
    {
      "role": "user",
      "content": [
        {
          "type": "text",
          "text": "Please convert the table in this image to three np.array(), where each item in each array is a tuple of the sol erda, sol erda fragment cost"
        },
        {
          "type": "image_url",
          "image_url": {
            "url": f"data:image/jpeg;base64,{base64_image}"
          }
        }
      ]
    }
  ],
  "max_tokens": 1500
}

response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload)

# print(response.json())
response.json()['choices'][0]['message']['content']


KeyError: 'choices'